# data 저장할 csv 파일 직접 만들기

In [35]:
import csv

csv_file_name = 'diary_data.csv'
# sample_data = [
#     {'datetime': '1111-11-11', 'texts': 'THIS IS A SAMPLE', 'sentiment': 'GUESS WHAT'}
# ]

try:
    with open(csv_file_name, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['datetime', 'texts', 'sentiment'])
        writer.writeheader()  # header(col)
        # for row in sample_data:
        #     writer.writerow(row)  # data가 있다면 row에 채워넣기
    print(f"{csv_file_name} saved!")
    
except Exception as e:
    print(f"Failed to create the file: {e}")

diary_data.csv saved!


# 사용할 함수들 정의 
    - diary 파일 읽기 
    - diary 파일 (새로) 저장
    - user texts 날짜와 함께 다이어리 [] 에 저장하기

In [66]:
import csv

def load_diary():
    diaries = []
    try:
        with open("diary_data.csv", mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                date = row['datetime']
                texts = row['texts']
                sentiment = row['sentiment']
                diaries.append({'datetime': date, 'texts': texts, 'sentiment': sentiment})
    except FileNotFoundError:
        print("No diary file found.")
    return diaries

def save_diary(diaries):
    try:
        with open("diary_data.csv", mode='w', newline='', encoding='utf-8') as file:
            fieldnames = ['datetime', 'texts', 'sentiment']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            for diary in diaries:
                writer.writerow(diary)
    except Exception as e:
        print(f"Failed to save the file: {e}")        

def get_user_texts(diaries):
    while True:
        date = input("What date is it today?\nYour input must follow this format exactly! >> 2024-04-05\n: ")
        texts = input("Tell me How your day was!\n: ")
        diaries.append({'datetime': date, 'texts': texts, 'sentiment': None})
        decision = input("If you want to stop updating your diary, then press 0\n: ")
        if decision == '0':
            return diaries

if __name__ == "__main__":
    diary = load_diary()
    updated_diary = get_user_texts(diary)
    save_diary(updated_diary)
    print("Diary successfully updated & saved!")

What date is it today?
Your input must follow this format exactly! >> 2024-04-05
:  2024-03-03
Tell me How your day was!
:  I HAD NOTHING TO DO..
If you want to stop updating your diary, then press 0
:  0


Diary successfully updated & saved!


In [67]:
updated_diary

[{'datetime': '1111-11-11',
  'texts': 'THIS IS A SAMPLE',
  'sentimental': 'GUESS WHAT'},
 {'datetime': '2024-04-03',
  'texts': 'I HAD NOTHING TO DO..',
  'sentimental': ''},
 {'datetime': '2024-03-03',
  'texts': 'I HAD NOTHING TO DO..',
  'sentimental': None}]

In [68]:
for one in updated_diary:
    print(one['texts'],'\n')
    
print(updated_diary[1]['texts'])

THIS IS A SAMPLE 

I HAD NOTHING TO DO.. 

I HAD NOTHING TO DO.. 

I HAD NOTHING TO DO..


# 감정 분석 어떤 모델 쓸지

In [69]:
from transformers import pipeline
sentiment_analyzer = pipeline('sentiment-analysis')  
def sentiment_analysis(text):
    return sentiment_analyzer(text)
analyze_emotions("My day was hectic but productive. Back-to-back meetings in the morning, a quick lunch at my desk, and then spent the afternoon working on a big project. Managed to squeeze in a workout before dinner. Now I'm winding down with a good book.")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'NEGATIVE', 'score': 0.9808839559555054}]

## 단순히 transformers만 가지고는 힘들어 보인다.<br>
https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest?text=Covid+cases+are+increasing+fast%21+But+i+know+and+i+hope+it+is+getting+way+better

In [ ]:
!pip install scipy

In [72]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)
text = "My day was hectic but productive. Back-to-back meetings in the morning, a quick lunch at my desk, and then spent the afternoon working on a big project. Managed to squeeze in a workout before dinner. Now I'm winding down with a good book."
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)
# text = "Covid cases are increasing fast!"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)
# Print labels and scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

C:\Neuer Ordner\envs\python3.9.16\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


1) positive 0.9236
2) neutral 0.071
3) negative 0.0055


In [79]:
text = "My day was busy but fulfilling. Got the kids ready for school, did some house chores, and ran a few errands. In the afternoon, I helped the kids with their homework and made dinner. Now I'm enjoying some quiet time with a cup of tea."
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 2)}")

1) positive 0.94
2) neutral 0.06
3) negative 0.0


In [80]:
text = "In the morning was busy but fulfilling. Got the kids ready for school, did some house chores, and ran a few errands. However in the afternoon, I helped the kids with their homework too much so that i don't feel now doing something at all."
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 2)}")

1) neutral 0.57
2) negative 0.23
3) positive 0.2


## 이거 좋다!
: This is a RoBERTa-base model trained on ~124M tweets from January 2018 to December 2021, and finetuned for sentiment analysis with the TweetEval benchmark.

# 감정분석 모델 & 위에서 구현 함수들 합치기

In [82]:
from transformers import AutoModelForSequenceClassification, TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
import os

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
LOCAL_MODEL_PATH = "twitter-roberta"

# local에 모델 저장해서 매번 허깅페이스에서 안불러오도록 함
if not os.path.exists(LOCAL_MODEL_PATH):
    os.makedirs(LOCAL_MODEL_PATH)
    
tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer.save_pretrained(LOCAL_MODEL_PATH)

config = AutoConfig.from_pretrained(MODEL)
config.save_pretrained(LOCAL_MODEL_PATH)

model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(LOCAL_MODEL_PATH)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
import csv, os
from transformers import AutoModelForSequenceClassification, TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from scipy.special import softmax
import numpy as np
import pandas as pd

def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

def load_model():
    LOCAL_MODEL_PATH = "twitter-roberta"
    tokenizer = AutoTokenizer.from_pretrained(LOCAL_MODEL_PATH)
    model = AutoModelForSequenceClassification.from_pretrained(LOCAL_MODEL_PATH)
    config = AutoConfig.from_pretrained(LOCAL_MODEL_PATH)
    return tokenizer, model, config

def load_diary():
    diaries = []
    try:
        with open("diary_data.csv", mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                date = row['datetime'] if row['datetime'] else None
                texts = row['texts'] if row['texts'] else None
                sentiment = row['sentiment'] if row['sentiment'] else None
                diaries.append({'datetime': date, 'texts': texts, 'sentiment': sentiment})
    except FileNotFoundError:
        print("No diary file found.")
    return diaries

def save_diary(diaries):
    try:
        with open("diary_data.csv", mode='w', newline='', encoding='utf-8') as file:
            fieldnames = ['datetime', 'texts', 'sentiment']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            for diary in diaries:
                writer.writerow(diary)
    except Exception as e:
        print(f"Failed to save the file: {e}")        

def sentiment_analysis(tokenizer, model, config, text): 
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    sentiment_scores = {config.id2label[ranking[i]]: np.round(float(scores[ranking[i]]), 2) for i in range(scores.shape[0])}
    # for i in range(scores.shape[0]):
    #     l = config.id2label[ranking[i]]
    #     s = scores[ranking[i]]
    #     print(f"{i+1}) {l} {np.round(float(s), 4)}")
    return sentiment_scores

def get_user_texts(diaries):
    while True:
        date = input("What date is it today?\nYour input must follow this format exactly! >> 2024-04-05\n: ")
        texts = input("Tell me How your day was!\n: ")
        sentiment_scores = sentiment_analysis(tokenizer, model, config, texts)
        diaries.append({'datetime': date, 'texts': texts, 'sentiment': sentiment_scores})
        decision = input("If you want to stop updating your diary, then press 0\n: ")
        if decision == '0':
            return diaries

if __name__ == "__main__":
    diary = load_diary()
    tokenizer, model, config = load_model()
    updated_diary = get_user_texts(diary)
    save_diary(updated_diary)
    print("Diary successfully updated & saved!")

What date is it today?
Your input must follow this format exactly! >> 2024-04-05
:  2024-05-05
Tell me How your day was!
:  My day was busy but fulfilling. Got the kids ready for school, did some house chores, and ran a few errands. In the afternoon, I helped the kids with their homework and made dinner. Now I'm enjoying some quiet time with a cup of tea.
If you want to stop updating your diary, then press 0
:  1
What date is it today?
Your input must follow this format exactly! >> 2024-04-05
:  2024-05-10
Tell me How your day was!
:  My day was busy. Got the kids ready for school, did some house chores, and ran a few errands. In the afternoon, I helped the kids with their homework and made dinner. Now finally I'm enjoying some quiet time with a cup of tea.
If you want to stop updating your diary, then press 0
:  202
What date is it today?
Your input must follow this format exactly! >> 2024-04-05
:  2024-05-02
Tell me How your day was!
:  My day was busy but fulfilling. Got the kids re

Diary successfully updated & saved!


# 날짜 최신순 정렬 + 정렬해야돼서 csv를 []말고 pandas로 읽어오는 것으로 변경

In [8]:
updated_diary

[{'datetime': '2024-05-05',
  'texts': "My day was busy but fulfilling. Got the kids ready for school, did some house chores, and ran a few errands. In the afternoon, I helped the kids with their homework and made dinner. Now I'm enjoying some quiet time with a cup of tea.",
  'sentiment': {'positive': 0.94, 'neutral': 0.06, 'negative': 0.0}},
 {'datetime': '2024-05-10',
  'texts': "My day was busy. Got the kids ready for school, did some house chores, and ran a few errands. In the afternoon, I helped the kids with their homework and made dinner. Now finally I'm enjoying some quiet time with a cup of tea.",
  'sentiment': {'positive': 0.8, 'neutral': 0.19, 'negative': 0.01}},
 {'datetime': '2024-05-02',
  'texts': "My day was busy but fulfilling. Got the kids ready for school, did some house chores, and ran a few errands. In the afternoon, I helped the kids with their homework and made dinner. Now I'm enjoying some quiet time with a cup of tea.",
  'sentiment': {'positive': 0.94, 'neut

In [14]:
diaries = pd.DataFrame(updated_diary)

In [15]:
from datetime import datetime
import pandas as pd

diaries['datetime'] = pd.to_datetime(diaries['datetime'])
diaries = diaries.sort_values(by='datetime', ascending=False) # 최신순 descending

In [21]:
diaries

,datetime,texts,sentiment
1,2024-05-10,My day was busy. Got the kids ready for school...,"{'positive': 0.8, 'neutral': 0.19, 'negative':..."
0,2024-05-05,My day was busy but fulfilling. Got the kids r...,"{'positive': 0.94, 'neutral': 0.06, 'negative'..."
2,2024-05-02,My day was busy but fulfilling. Got the kids r...,"{'positive': 0.94, 'neutral': 0.06, 'negative'..."


In [20]:
print(diaries.dtypes) # datetime type이 datetime인 것을 확인

datetime     datetime64[ns]
texts                object
sentiment            object
dtype: object


In [48]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

def load_model():
    LOCAL_MODEL_PATH = "twitter-roberta"
    tokenizer = AutoTokenizer.from_pretrained(LOCAL_MODEL_PATH)
    model = AutoModelForSequenceClassification.from_pretrained(LOCAL_MODEL_PATH)
    config = AutoConfig.from_pretrained(LOCAL_MODEL_PATH)
    return tokenizer, model, config

def load_diary():
    try:
        diaries = pd.read_csv("diary_data.csv", parse_dates=['datetime'])
    except FileNotFoundError: # 내용이 없어도 만들어야 함
        diaries = pd.DataFrame(columns=['datetime', 'texts', 'sentiment'])
    return diaries

def save_diary(diaries):
    try:
        diaries.to_csv("diary_data.csv", index=False, encoding='utf-8')
    except Exception as e:
        print(f"Failed to save the file: {e}")    

def sort_diary(diaries):
    diaries['datetime'] = pd.to_datetime(diaries['datetime'], format="%Y-%m-%d") # datetime 객체로 변환
    diaries = diaries.sort_values(by='datetime', ascending=False) # 최신순, descending
    return diaries

def sentiment_analysis(tokenizer, model, config, text): 
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    sentiment_scores = {config.id2label[ranking[i]]: np.round(float(scores[ranking[i]]), 2) for i in range(scores.shape[0])}
    return sentiment_scores

def get_user_texts(diaries):
    while True:
        date = input("What date is it today?\nYour input must follow this format exactly! >> 2024-04-05\n: ")
        texts = input("Tell me How your day was!\n: ")
        sentiment_scores = sentiment_analysis(tokenizer, model, config, texts)
        new_diary = pd.DataFrame([{'datetime': date, 'texts': texts, 'sentiment': sentiment_scores}])
        diaries = pd.concat([diaries, new_diary], ignore_index=True)
        diaries = sort_diary(diaries)
        decision = input("If you want to stop updating your diary, then press 0\n: ")
        if decision == '0':
            return diaries

if __name__ == "__main__":
    diary = load_diary()
    tokenizer, model, config = load_model()
    updated_diary = get_user_texts(diary)
    save_diary(updated_diary)
    print("Diary successfully updated & saved!")

What date is it today?
Your input must follow this format exactly! >> 2024-04-05
:  2024-05-22
Tell me How your day was!
:  My day was hectic but productive. Back-to-back meetings in the morning, a quick lunch at my desk, and then spent the afternoon working on a big project. Managed to squeeze in a workout before dinner. Now I'm winding down with a good book.
If you want to stop updating your diary, then press 0
:  0


Diary successfully updated & saved!


In [49]:
updated_diary

,datetime,texts,sentiment
3,2024-05-22,My day was hectic but productive. Back-to-back...,"{'positive': 0.92, 'neutral': 0.07, 'negative'..."
2,2024-05-06,Oh.. I don't really feel going to school today...,"{'negative': 0.53, 'neutral': 0.28, 'positive'..."
1,2024-05-05,Yayy!! Children's day!! I have a lot to say ab...,"{'positive': 0.98, 'neutral': 0.02, 'negative'..."
0,2024-05-02,My day was busy but fulfilling. Got the kids r...,"{'positive': 0.94, 'neutral': 0.06, 'negative'..."


In [50]:
print(updated_diary['datetime'].head())
print(updated_diary['datetime'].dtype)

3   2024-05-22
2   2024-05-06
1   2024-05-05
0   2024-05-02
Name: datetime, dtype: datetime64[ns]
datetime64[ns]


# text-to-image 생성 및 저장

In [51]:
from diffusers import DiffusionPipeline
import torch

# load both base & refiner
base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
base.to("cuda")
refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)
refiner.to("cuda")

# Define how many steps and what % of steps to be run on each experts (80/20) here
n_steps = 40
high_noise_frac = 0.8

prompt = "A majestic lion jumping from a big stone at night"

# run both experts
image = base(
    prompt=prompt,
    num_inference_steps=n_steps,
    denoising_end=high_noise_frac,
    output_type="latent",
).images
image = refiner(
    prompt=prompt,
    num_inference_steps=n_steps,
    denoising_start=high_noise_frac,
    image=image,
).images[0]

C:\Neuer Ordner\envs\python3.9.16\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

C:\Neuer Ordner\envs\python3.9.16\Lib\site-packages\huggingface_hub\file_download.py:982: UserWarning: Not enough free disk space to download the file. The expected file size is: 5135.15 MB. The target location C:\Users\황의지\.cache\huggingface\hub\models--stabilityai--stable-diffusion-xl-base-1.0\blobs only has 0.00 MB free disk space.
  warnings.warn(


diffusion_pytorch_model.fp16.safetensors:  45%|####5     | 2.33G/5.14G [00:00<?, ?B/s]

OSError: [Errno 28] No space left on device

In [ ]:
from diffusers import DiffusionPipeline
import torch

def save_models():
    base_model = "stabilityai/stable-diffusion-xl-base-1.0"
    refiner_model = "stabilityai/stable-diffusion-xl-refiner-1.0"
    local_base_path = "table_diffusion_xl_base"
    local_refiner_path = "stable_diffusion_xl_refiner"
    
    base = DiffusionPipeline.from_pretrained(
        base_model, torch_dtype=torch.float16, variant="fp16", use_safetensors=True
    )
    base.save_pretrained(local_base_path)
    
    refiner = DiffusionPipeline.from_pretrained(
        refiner_model_name,
        text_encoder_2=base.text_encoder_2,
        vae=base.vae,
        torch_dtype=torch.float16,
        use_safetensors=True,
        variant="fp16",
    )
    refiner.save_pretrained(local_refiner_path)

save_models()

In [ ]:
from diffusers import DiffusionPipeline
import torch

def load_models(prompt, n_steps=40, high_noise_frac=0.8):
    local_base_path = "stable_diffusion_xl_base"
    local_refiner_path = "stable_diffusion_xl_refiner"

    # Load base model from local path
    base = DiffusionPipeline.from_pretrained(
        local_base_path, torch_dtype=torch.float16, variant="fp16", use_safetensors=True
    )
    base.to("cuda")

    # Load refiner model from local path
    refiner = DiffusionPipeline.from_pretrained(
        local_refiner_path,
        text_encoder_2=base.text_encoder_2,
        vae=base.vae,
        torch_dtype=torch.float16,
        use_safetensors=True,
        variant="fp16",
    )
    refiner.to("cuda")

    # Run both models to generate the image
    image = base(
        prompt=prompt,
        num_inference_steps=n_steps,
        denoising_end=high_noise_frac,
        output_type="latent",
    ).images

    image = refiner(
        prompt=prompt,
        num_inference_steps=n_steps,
        denoising_start=high_noise_frac,
        image=image,
    ).images[0]

    return image

# Example usage
prompt = "A majestic lion jumping from a big stone at night"
image = load_models(prompt)
image.save("generated_image.png")

## 저장하기엔.. 컴퓨터 용량이 없다 (매번 GPU를 못쓰니까..)<br>

In [ ]:
from diffusers import DiffusionPipeline
import torch

# load both base & refiner
base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
base.to("cuda")
refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)
refiner.to("cuda")

# Define how many steps and what % of steps to be run on each experts (80/20) here
n_steps = 40
high_noise_frac = 0.8

prompt = "Had a relaxing day. Went for a walk in the park in the morning, did some gardening, and read a few chapters of a novel. In the afternoon, I met friends for coffee and now I'm watching a movie at home."

# run both experts
image = base(
    prompt=prompt,
    num_inference_steps=n_steps,
    denoising_end=high_noise_frac,
    output_type="latent",
).images
image = refiner(
    prompt=prompt,
    num_inference_steps=n_steps,
    denoising_start=high_noise_frac,
    image=image,
).images[0]

1. **A Busy Professional**:
   "My day was hectic but productive. Back-to-back meetings in the morning, a quick lunch at my desk, and then spent the afternoon working on a big project. Managed to squeeze in a workout before dinner. Now I'm winding down with a good book."

2. **A College Student**:
   "Today was pretty good. Had a couple of lectures in the morning, grabbed lunch with friends, and then headed to the library to study for an upcoming exam. In the evening, I went to a campus event and now I'm relaxing with some Netflix."

3. **A Stay-at-Home Parent**:
   "My day was busy but fulfilling. Got the kids ready for school, did some house chores, and ran a few errands. In the afternoon, I helped the kids with their homework and made dinner. Now I'm enjoying some quiet time with a cup of tea."

4. **A Freelance Writer**:
   "Today was creatively satisfying. Spent the morning brainstorming ideas and the afternoon writing an article for a client. Took a break to walk my dog and now I'm finishing up some edits before calling it a day."

5. **A Retiree**:
   "Had a relaxing day. Went for a walk in the park in the morning, did some gardening, and read a few chapters of a novel. In the afternoon, I met friends for coffee and now I'm watching a movie at home."

1. **Busy Professional**: "Hectic day with meetings and project work. Managed a workout and now relaxing with a book."

2. **College Student**: "Lectures, lunch with friends, library study session, campus event, and Netflix to end the day."

3. **Stay-at-Home Parent**: "Kids to school, errands, house chores, homework help, dinner, and some quiet tea time."

4. **Freelance Writer**: "Brainstormed in the morning, wrote in the afternoon, walked the dog, and finished edits."

5. **Retiree**: "Walked in the park, did some gardening, met friends for coffee, and now watching a movie."

In [12]:
# gpt2 때는 아직 meta-learning이 아니어서 그런듯

from transformers import GPT2Tokenizer, GPT2LMHeadModel

# 모델과 토크나이저 불러오기
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

users_texts = "Brainstormed in the morning, wrote in the afternoon, walked the dog, and finished edits."
# 프롬프트 설정
prompt = f"You are a diary writer. You should create long diary texts referring users-texts. The users-texts is like \"{users_texts}\". Generate Diary\n"

# 프롬프트를 토큰화
inputs = tokenizer.encode(prompt, return_tensors="pt")

# 텍스트 생성 시작
output_sequences = model.generate(
    input_ids=inputs,
    max_length=300,  # 최대 길이 설정
    temperature=0.7,  # 생성 다양성을 위한 온도 설정
    top_k=50,  # 가장 높은 k 확률을 가진 토큰들만 고려
)

# 생성된 텍스트 디코딩
generated_text = tokenizer.decode(output_sequences[0], clean_up_tokenization_spaces=True)

print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


You are a diary writer. You should create long diary texts referring users-texts. The users-texts is like "Brainstormed in the morning, wrote in the afternoon, walked the dog, and finished edits.". Generate Diary

The first step is to create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diary. Create a diar

# 다이어리 수정/삭제 관리

# 감정 월별/일별 평균 내기

In [ ]:
pip install diffusers --upgrade

In [ ]:
pip install invisible_watermark transformers accelerate safetensors